# **BOUTIQUE HOTEL PERFORMANCE DATA CLEANING**

## **1. Load Data**

In [59]:
# Import Libraries
import pandas as pd
import numpy as np

# Load Data
customers_raw = pd.read_csv("customers.csv")
orders_raw = pd.read_csv("orders.csv")
rooms_raw = pd.read_csv("rooms.csv")

# Make Copies for Cleaning
customers = customers_raw.copy()
orders = orders_raw.copy()
rooms = rooms_raw.copy()

## **2. Initial Data Inspection**

In [60]:
# INITIAL DATA INSPECTION

datasets = {
    "Customers": customers,
    "Orders": orders,
    "Rooms": rooms
}

for name, df in datasets.items():
    print(f"\n{'='*40}")
    print(f"{name} Table")
    print(f"{'='*40}")
    print("Shape:", df.shape)
    print("\nInfo:")
    df.info()
    print("\nPreview:")
    display(df.head())


Customers Table
Shape: (7188, 4)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7188 entries, 0 to 7187
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CustomerID       7188 non-null   int64 
 1   CustomerAge      7188 non-null   int64 
 2   CustomerGender   7188 non-null   object
 3   CustomerCountry  7188 non-null   object
dtypes: int64(2), object(2)
memory usage: 224.8+ KB

Preview:


,CustomerID,CustomerAge,CustomerGender,CustomerCountry
0,1,72,Male,Germany
1,2,32,Male,Sweden
2,3,70,Female,Turkey
3,4,68,Female,Sweden
4,5,80,Male,Sweden



Orders Table
Shape: (5293, 10)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5293 entries, 0 to 5292
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   OrderID         5293 non-null   int64  
 1   CustomerIDs     5293 non-null   object 
 2   CustomerCount   5293 non-null   int64  
 3   RoomNumber      5293 non-null   int64  
 4   CheckInDate     5293 non-null   object 
 5   CheckOutDate    5293 non-null   object 
 6   StayDuration    5293 non-null   int64  
 7   TotalCost       5293 non-null   float64
 8   PaymentMethod   5293 non-null   object 
 9   SeasonalFactor  5293 non-null   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 413.6+ KB

Preview:


,OrderID,CustomerIDs,CustomerCount,RoomNumber,CheckInDate,CheckOutDate,StayDuration,TotalCost,PaymentMethod,SeasonalFactor
0,1,"1,2",2,117,2024-01-01,2024-01-02,1,143.23,Online Transfer,normal
1,2,3,1,105,2024-01-01,2024-01-02,1,98.05,Credit Card,normal
2,3,"4,5",2,112,2024-01-01,2024-01-02,1,146.95,Credit Card,normal
3,4,6,1,104,2024-01-01,2024-01-02,1,102.13,Online Transfer,normal
4,5,7,1,103,2024-01-01,2024-01-02,1,97.21,Online Transfer,normal



Rooms Table
Shape: (20, 4)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   RoomNumber  20 non-null     int64 
 1   Capacity    20 non-null     int64 
 2   RoomType    20 non-null     object
 3   BasePrice   20 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 772.0+ bytes

Preview:


,RoomNumber,Capacity,RoomType,BasePrice
0,101,1,Single Room,100
1,102,1,Single Room,100
2,103,1,Single Room,100
3,104,1,Single Room,100
4,105,1,Single Room,100


## **3. Handling Missing Values**

In [61]:
# DOUBLE CHECK MISSING VALUES FOR ALL TABLES

datasets = {
    "Customers": customers,
    "Orders": orders,
    "Rooms": rooms
}

for name, df in datasets.items():
    print(f"\n{name} - Missing Values by Column:")
    print(df.isnull().sum())


Customers - Missing Values by Column:
CustomerID         0
CustomerAge        0
CustomerGender     0
CustomerCountry    0
dtype: int64

Orders - Missing Values by Column:
OrderID           0
CustomerIDs       0
CustomerCount     0
RoomNumber        0
CheckInDate       0
CheckOutDate      0
StayDuration      0
TotalCost         0
PaymentMethod     0
SeasonalFactor    0
dtype: int64

Rooms - Missing Values by Column:
RoomNumber    0
Capacity      0
RoomType      0
BasePrice     0
dtype: int64


The dataset contains no null or missing values, indicating that no action for missing data was required.

## **4. Normalize Data**

In [62]:
# NORMALIZE ORDERS: ONE ROW PER CUSTOMER

# Split the CustomerIDs string into a list
orders['CustomerIDs'] = orders['CustomerIDs'].str.split(',')

# Explode the list into separate rows
orders = orders.explode('CustomerIDs')

# Convert CustomerIDs to int
orders['CustomerIDs'] = orders['CustomerIDs'].astype(int)

# Rename column to CustomerID
orders.rename(columns={'CustomerIDs': 'CustomerID'}, inplace=True)

# Reset index
orders.reset_index(drop=True, inplace=True)

# Preview
display(orders.sample(10))


,OrderID,CustomerID,CustomerCount,RoomNumber,CheckInDate,CheckOutDate,StayDuration,TotalCost,PaymentMethod,SeasonalFactor
4340,3213,4341,4,120,2024-08-04,2024-08-06,2,495.73,Credit Card,normal
3862,2861,3863,2,117,2024-07-11,2024-07-12,1,146.46,Cash,normal
4930,3648,4931,2,112,2024-09-06,2024-09-09,3,217.43,Online Transfer,low_season
6285,4653,6286,1,117,2024-11-17,2024-11-19,2,290.77,Credit Card,normal
5777,4289,5778,2,111,2024-10-23,2024-10-24,1,153.67,Cash,normal
641,495,642,2,112,2024-02-04,2024-02-06,2,310.80,Credit Card,normal
1224,924,1225,2,114,2024-03-04,2024-03-06,2,470.75,Credit Card,high_season
2844,2120,2845,1,104,2024-05-25,2024-06-01,7,669.34,Credit Card,normal
2017,1504,2018,2,111,2024-04-11,2024-04-13,2,289.08,Cash,normal
3006,2237,3007,1,107,2024-06-02,2024-06-03,1,150.77,Online Transfer,high_season


## **5. Converting Incorrect Data Type**

In [63]:
# CONVERT INCORRECT OBJECT TYPE INTO DATE TYPE

# Convert CheckInDate and CheckOutDate to datetime
orders["CheckInDate"] = pd.to_datetime(
    orders["CheckInDate"].astype(str).str.strip(), errors="coerce"
)
orders["CheckOutDate"] = pd.to_datetime(
    orders["CheckOutDate"].astype(str).str.strip(), errors="coerce"
)

# Convert BasePrice to numeric and ensure float type
rooms["BasePrice"] = pd.to_numeric(rooms["BasePrice"], errors="coerce").astype(float)

In [64]:
# CHECK DATA TYPE AFTER CORRECTING

print(orders.info())
print(rooms.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7188 entries, 0 to 7187
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OrderID         7188 non-null   int64         
 1   CustomerID      7188 non-null   int64         
 2   CustomerCount   7188 non-null   int64         
 3   RoomNumber      7188 non-null   int64         
 4   CheckInDate     7188 non-null   datetime64[ns]
 5   CheckOutDate    7188 non-null   datetime64[ns]
 6   StayDuration    7188 non-null   int64         
 7   TotalCost       7188 non-null   float64       
 8   PaymentMethod   7188 non-null   object        
 9   SeasonalFactor  7188 non-null   object        
dtypes: datetime64[ns](2), float64(1), int64(5), object(2)
memory usage: 561.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----

## **6. Handling Duplicate Data**

In [65]:
# CHECK DUPLICATE DATA FOR ALL TABLES

datasets = {
    "Customers": customers,
    "Orders": orders,
    "Rooms": rooms
}

for name, df in datasets.items():
    duplicates = df.duplicated().sum()
    print(f"{name} - Number of duplicate rows: {duplicates}")


Customers - Number of duplicate rows: 0
Orders - Number of duplicate rows: 0
Rooms - Number of duplicate rows: 0


No duplicate data were found across all datasets. Therefore, no action was required.

In [66]:
# CHECK DUPLICATES DATA IN PRIMARY KEYS

pk_columns = {
    "Orders": ("orders", "OrderID"),
    "Customers": ("customers", "CustomerID"),
    "Rooms": ("rooms", "RoomNumber")
}

for table_name, (df_name, col) in pk_columns.items():
    print(f"\n=== Checking {col} in {table_name} ===")
    df = globals()[df_name]  # get the dataframe by name
    dup_count = df[col].duplicated().sum()
    print(f"Duplicate count: {dup_count}")



=== Checking OrderID in Orders ===
Duplicate count: 1895

=== Checking CustomerID in Customers ===
Duplicate count: 0

=== Checking RoomNumber in Rooms ===
Duplicate count: 0


In [67]:
# DROP DUPLICATES IN PRIMARY KEY

# Keep only the first customer per OrderID
orders = orders.drop_duplicates(subset=['OrderID'], keep='first')

# Reset index
orders.reset_index(drop=True, inplace=True)

# Preview
print(f"Duplicate data in OrderID: {orders['OrderID'].duplicated().sum()}")
display(orders.head(10))


Duplicate data in OrderID: 0


,OrderID,CustomerID,CustomerCount,RoomNumber,CheckInDate,CheckOutDate,StayDuration,TotalCost,PaymentMethod,SeasonalFactor
0,1,1,2,117,2024-01-01,2024-01-02,1,143.23,Online Transfer,normal
1,2,3,1,105,2024-01-01,2024-01-02,1,98.05,Credit Card,normal
2,3,4,2,112,2024-01-01,2024-01-02,1,146.95,Credit Card,normal
3,4,6,1,104,2024-01-01,2024-01-02,1,102.13,Online Transfer,normal
4,5,7,1,103,2024-01-01,2024-01-02,1,97.21,Online Transfer,normal
5,6,8,1,106,2024-01-01,2024-01-05,4,410.36,Cash,normal
6,7,9,2,118,2024-01-01,2024-01-03,2,399.43,Credit Card,normal
7,8,11,2,116,2024-01-01,2024-01-05,4,578.17,Credit Card,normal
8,9,13,1,109,2024-01-01,2024-01-06,5,486.45,Credit Card,normal
9,10,14,2,111,2024-01-01,2024-01-04,3,450.42,Online Transfer,normal


## **7. Handling Inconsistent Data**

In [68]:
# CHECK CATEGORICAL VALUE CONSISTENCY FOR ALL TABLES

datasets = {
    "Customers": customers,
    "Orders": orders,
    "Rooms": rooms
}

print("=== CATEGORICAL VALUE CONSISTENCY CHECK ===")
for name, df in datasets.items():
    cat_cols = df.select_dtypes(include="object").columns
    if len(cat_cols) == 0:
        print(f"\n{name}: No categorical columns")
        continue

    print(f"\n{name}:")
    for col in cat_cols:
        unique_values = df[col].str.strip().unique()
        print(f"  {col}: {unique_values}")

=== CATEGORICAL VALUE CONSISTENCY CHECK ===

Customers:
  CustomerGender: ['Male' 'Female' 'Other']
  CustomerCountry: ['Germany' 'Sweden' 'Turkey' 'Netherlands' 'Gibraltar' 'Zambia'
 'Guinea-Bissau' 'Sao Tome and Principe' 'Faroe Islands' 'Congo'
 'United Arab Emirates' 'French Southern Territories' 'Lithuania'
 'Russian Federation' 'Central African Republic' 'Cook Islands' 'Vanuatu'
 'Dominican Republic' 'Liechtenstein' 'Australia' 'Iceland' 'India'
 'Canada' 'Saudi Arabia' 'Trinidad and Tobago' 'Egypt' 'Cayman Islands'
 'Zimbabwe' 'Chile' 'Saint Kitts and Nevis' 'Norfolk Island' 'Niue'
 'Lebanon' 'Northern Mariana Islands' 'Tokelau' 'Kuwait' 'Portugal'
 'Sudan' 'Cuba' "Lao People's Democratic Republic" 'Aruba' 'Turkmenistan'
 'United States of America' 'Kyrgyz Republic' 'Angola' 'Ethiopia'
 'Ukraine' 'Tunisia' 'Guatemala' 'Nigeria' 'Chad' 'Korea' 'Honduras'
 'Jordan' 'Somalia' 'Puerto Rico' 'Nauru' 'Pitcairn Islands' 'Montenegro'
 'Mali' 'Saint Pierre and Miquelon' 'Mauritania' 'Sou

In [69]:
# FOREIGN KEY CHECKS

# 1. CUSTOMERS (orders → customers)
print("\n=== FOREIGN KEY CHECK: CUSTOMER ===")
invalid_customer = (~orders["CustomerID"].isin(customers["CustomerID"])).sum()
print(f"Invalid CustomerID references in orders: {invalid_customer}")

# 2. ROOMS (orders → rooms)
print("\n=== FOREIGN KEY CHECK: ROOM ===")
invalid_room = (~orders["RoomNumber"].isin(rooms["RoomNumber"])).sum()
print(f"Invalid RoomNumber references in orders: {invalid_room}")



=== FOREIGN KEY CHECK: CUSTOMER ===
Invalid CustomerID references in orders: 0

=== FOREIGN KEY CHECK: ROOM ===
Invalid RoomNumber references in orders: 0


In [70]:
# LOGICAL / RANGE CHECKS

# 1. CUSTOMERS

print("\n=== CUSTOMERS TABLE ===")
# Age should be reasonable (e.g., 18–80)
invalid_age = (~customers["CustomerAge"].between(18, 80)).sum()
print(f"Invalid CustomerAge (outside 18–80): {invalid_age}")

# 2. ORDERS

print("\n=== ORDERS TABLE ===")
# StayDuration should be positive
invalid_stay = (orders["StayDuration"] <= 0).sum()
print(f"Invalid StayDuration (<= 0): {invalid_stay}")

# TotalCost should be positive
invalid_totalcost = (orders["TotalCost"] <= 0).sum()
print(f"Invalid TotalCost (<= 0): {invalid_totalcost}")

# CustomerCount should be positive and not exceed room capacity (optional check)
invalid_custcount = (orders["CustomerCount"] <= 0).sum()
print(f"Invalid CustomerCount (<= 0): {invalid_custcount}")

# CheckInDate should be before CheckOutDate
invalid_dates = (orders["CheckInDate"] >= orders["CheckOutDate"]).sum()
print(f"Invalid date ranges (CheckInDate >= CheckOutDate): {invalid_dates}")

# 3. ROOMS

print("\n=== ROOMS TABLE ===")
# Capacity should be positive
invalid_capacity = (rooms["Capacity"] <= 0).sum()
print(f"Invalid Room Capacity (<= 0): {invalid_capacity}")

# BasePrice should be positive
invalid_baseprice = (rooms["BasePrice"] <= 0).sum()
print(f"Invalid Room BasePrice (<= 0): {invalid_baseprice}")



=== CUSTOMERS TABLE ===
Invalid CustomerAge (outside 18–80): 0

=== ORDERS TABLE ===
Invalid StayDuration (<= 0): 0
Invalid TotalCost (<= 0): 0
Invalid CustomerCount (<= 0): 0
Invalid date ranges (CheckInDate >= CheckOutDate): 0

=== ROOMS TABLE ===
Invalid Room Capacity (<= 0): 0
Invalid Room BasePrice (<= 0): 0


## **8. Final Data Inspection**

In [71]:
# FINAL DATA INSPECTION

datasets = {
    "Customers": customers,
    "Orders": orders,
    "Rooms": rooms
}

for name, df in datasets.items():
    print(f"\n{'='*40}")
    print(f"{name} Table")
    print(f"{'='*40}")
    print("Shape:", df.shape)
    print("\nInfo:")
    df.info()
    print("\nPreview:")
    display(df.head())


Customers Table
Shape: (7188, 4)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7188 entries, 0 to 7187
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CustomerID       7188 non-null   int64 
 1   CustomerAge      7188 non-null   int64 
 2   CustomerGender   7188 non-null   object
 3   CustomerCountry  7188 non-null   object
dtypes: int64(2), object(2)
memory usage: 224.8+ KB

Preview:


,CustomerID,CustomerAge,CustomerGender,CustomerCountry
0,1,72,Male,Germany
1,2,32,Male,Sweden
2,3,70,Female,Turkey
3,4,68,Female,Sweden
4,5,80,Male,Sweden



Orders Table
Shape: (5293, 10)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5293 entries, 0 to 5292
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OrderID         5293 non-null   int64         
 1   CustomerID      5293 non-null   int64         
 2   CustomerCount   5293 non-null   int64         
 3   RoomNumber      5293 non-null   int64         
 4   CheckInDate     5293 non-null   datetime64[ns]
 5   CheckOutDate    5293 non-null   datetime64[ns]
 6   StayDuration    5293 non-null   int64         
 7   TotalCost       5293 non-null   float64       
 8   PaymentMethod   5293 non-null   object        
 9   SeasonalFactor  5293 non-null   object        
dtypes: datetime64[ns](2), float64(1), int64(5), object(2)
memory usage: 413.6+ KB

Preview:


,OrderID,CustomerID,CustomerCount,RoomNumber,CheckInDate,CheckOutDate,StayDuration,TotalCost,PaymentMethod,SeasonalFactor
0,1,1,2,117,2024-01-01,2024-01-02,1,143.23,Online Transfer,normal
1,2,3,1,105,2024-01-01,2024-01-02,1,98.05,Credit Card,normal
2,3,4,2,112,2024-01-01,2024-01-02,1,146.95,Credit Card,normal
3,4,6,1,104,2024-01-01,2024-01-02,1,102.13,Online Transfer,normal
4,5,7,1,103,2024-01-01,2024-01-02,1,97.21,Online Transfer,normal



Rooms Table
Shape: (20, 4)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RoomNumber  20 non-null     int64  
 1   Capacity    20 non-null     int64  
 2   RoomType    20 non-null     object 
 3   BasePrice   20 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 772.0+ bytes

Preview:


,RoomNumber,Capacity,RoomType,BasePrice
0,101,1,Single Room,100.0
1,102,1,Single Room,100.0
2,103,1,Single Room,100.0
3,104,1,Single Room,100.0
4,105,1,Single Room,100.0


## **9. Save Cleaned Data**

In [ ]:
# SAVE CLEANED DATASETS

customers.to_csv("customers.csv", index=False)
orders.to_csv("orders.csv", index=False)
rooms.to_csv("rooms.csv", index=False)
